Holder Table Function 
=====================


In [1]:
using Pkg
Pkg.activate("../../.")
using Globtim
# include("../../../src/lib_func.jl") # Include the library of functions

# Constants and Parameters
d = 2 # Initial Degree 
const n, a, b = 2, 10, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .5 , 1 / 10  # Sampling parameters
const tol_l2 = 1.5e-2            # Define the tolerance for the L2-norm

f = HolderTable # Objective function

  Activating project at `~/Globtim.jl`


HolderTable (generic function with 1 method)

In [2]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;

current L2-norm: 0.3225768101075613
Number of samples: 8
Current degree: 2
current L2-norm: 0.34164522984081835
Number of samples: 12
Current degree: 3
current L2-norm: 0.15420669778631205
Number of samples: 17
Current degree: 4
current L2-norm: 0.16150972067976094
Number of samples: 22
Current degree: 5
current L2-norm: 0.10982203454764894
Number of samples: 28
Current degree: 6
current L2-norm: 0.09002980177601798
Number of samples: 34
Current degree: 7
current L2-norm: 0.07219154289178054
Number of samples: 41
Current degree: 8
current L2-norm: 0.061872080671082644
Number of samples: 48
Current degree: 9
current L2-norm: 0.04247168178265791
Number of samples: 56
Current degree: 10
current L2-norm: 0.03650326901605908
Number of samples: 65
Current degree: 11
current L2-norm: 0.026535168562028325
Number of samples: 74
Current degree: 12
current L2-norm: 0.02370545610011522
Number of samples: 83
Current degree: 13
current L2-norm: 0.01834622863611993
Number of samples: 93
Current degre

In [3]:
using DynamicPolynomials, DataFrames
using HomotopyContinuation, ProgressLogging

@polyvar(x[1:n]) # Define polynomial ring 
pol = main_nd(x, n, d, poly_approx.coeffs) 
grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d-1)

The system is of degree: 15


In [4]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false);

Computing mixed cells... 2    Time: 0:00:00
Computing mixed cells... 18    Time: 0:00:00
  mixed_volume:  225


Tracking 225 paths...   1%|▎                            |  ETA: 0:28:42



Tracking 225 paths... 100%|█████████████████████████████| Time: 0:00:16
  # paths tracked:                  225
  # non-singular solutions (real):  225 (189)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         225 (189)


In [5]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

In [6]:
using GLMakie

# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
z_coords = poly_approx.z

if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with a darker core
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black, label="Sampled Data Core")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=4, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red, label="Exact approximant critical points")

    display(fig)
end

GLMakie.Screen(...)

In [7]:

coords = poly_approx.scale_factor * poly_approx.grid # Extract coordinates and function values
z_coords = poly_approx.z

if size(coords)[2] == 2 # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Holder Table Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with darker contrast
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=:black, colormap=:viridis, label="Sampled Data")
    # Adjust the size and color of the sampled data points
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=5, color=z_coords, colormap=:viridis, label="Sampled Data")

    display(fig)
end

GLMakie.Screen(...)

In [8]:

# Create a grid for the surface plot
x1 = range(-1, 1, length=100)
y = range(-1, 1, length=100)
z = [f([xi, yi]) for xi in x1, yi in y]

fig = Figure(size=(800, 600))
ax = Axis3(fig[1, 1], title="3D Plot of Shubert function with the critical points of the approximant of degree 22",
           xlabel="x", ylabel="y", zlabel="f(x, y)")

# Plot the surface
surface!(ax, x1, y, z, color=(:white, 0.25), transparency=true)
# map=:(viridis,.)

# Plot the critical points
scatter!(ax, df.x, df.y, df.z, markersize=5, color=:red, label="Critical Points")

fig

Should plot the polynomial approximant too.

In [9]:
# Update tolerance
const tol_l2 = 1.5e-3

# Re-run the main computation loop
d = 2 # Reset degree
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        println("Current degree: ", d)
        global d += 1
    end
end;

# Re-run the polynomial system setup and solve
pol = main_nd(x, n, d, poly_approx.coeffs) 
grad = differentiate.(pol, x)
sys = System(grad)
println("The system is of degree: ", d-1)

Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false)

# Filter points and create DataFrame
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Collect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z=h_z); # Create a DataFrame

current L2-norm: 0.3225768101075613


Number of samples: 8
Current degree: 2
current L2-norm: 0.34164522984081835
Number of samples: 12
Current degree: 3
current L2-norm: 0.15420669778631205
Number of samples: 17
Current degree: 4
current L2-norm: 0.16150972067976094
Number of samples: 22
Current degree: 5
current L2-norm: 0.10982203454764894
Number of samples: 28
Current degree: 6
current L2-norm: 0.09002980177601798
Number of samples: 34
Current degree: 7
current L2-norm: 0.07219154289178054
Number of samples: 41
Current degree: 8
current L2-norm: 0.061872080671082644
Number of samples: 48
Current degree: 9
current L2-norm: 0.04247168178265791
Number of samples: 56
Current degree: 10
current L2-norm: 0.03650326901605908
Number of samples: 65
Current degree: 11
current L2-norm: 0.026535168562028325
Number of samples: 74
Current degree: 12
current L2-norm: 0.02370545610011522
Number of samples: 83
Current degree: 13
current L2-norm: 0.01834622863611993
Number of samples: 93
Current degree: 14
current L2-norm: 0.0166368349

In [10]:
# Plot using GLMakie

# Extract coordinates and function values
coords = poly_approx.scale_factor * poly_approx.grid
z_coords = poly_approx.z

if size(coords)[2] == 2  # Plot the 3D scatter plot if the dimensions are 2
    fig = Figure(size=(800, 600))
    ax = Axis3(fig[1, 1], title="Mixture of Gaussians Sample Points", xlabel="X-axis", ylabel="Y-axis", zlabel="Z-axis")

    # Plot the sampled data with adjusted size and color to create a halo effect
    scatter!(ax, coords[:, 1], coords[:, 2], z_coords, markersize=2, color=z_coords, colormap=:viridis, label="Sampled Data Halo")

    # Plot the exact approximant critical points with adjusted size
    scatter!(ax, df.x, df.y, df.z, markersize=10, color=:red, label="Exact approximant critical points")

    display(fig)
end

GLMakie.Screen(...)